In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

from helper import groupby_consecutive

import sys

sys.path.append("..")
from plotly_resampler import FigureResampler, FigureWidgetResampler, EveryNthPoint


# Adding dynamic aggregation to your plotly Figure

In [ ]:
# Some dummy data that will be used throughout the examples
n = 2_000_000
x = np.arange(n)
noisy_sine = (3 + np.sin(x / 2000) + np.random.randn(n) / 10) * x / 1000


## **auto mode**: `register_plotly_resampler`

Once `register_plotly_resampler` method is called, it will automatically convert all new defined plotly graph objects into a `FigureResampler` or `FigureWidgetResampler` object. The `mode` parameter of this method allows to define which type of the aforementioned resampling objects is used.

✅ **advantages**:
* This is the most convenient way to make your existing codebase more scalable
* You can keep on using your regular graph construction code

❌ **disadvantages**:
* all figures will be wrapped with plotly-resampler behavior (less control)
* if you want to go the extra mile regarding graph construction performance, it is better to use the `FigureResampler` or `FigureWidgetResampler` components individually.

In [ ]:
from plotly_resampler import register_plotly_resampler, unregister_plotly_resampler


In [ ]:
# by default, 1,000 samples per trace are shown
register_plotly_resampler(mode="auto", default_n_shown_samples=4500)


In [ ]:
# auto mode: when working in an IPython environment, this will automatically be a
# FigureWidgetResampler else, this will be an FigureResampler
fig = go.Figure()
fig.add_trace({"y": noisy_sine + 2, "name": "yp2", "showlegend": True})
print(type(fig))
fig


❗ `.show()` always returns a static html view

In [ ]:
# this outputs a static html view of the figure, which can be serialized within notebooks
fig.show()


### pro tip: `register_plotly_resampler` + pandas plotting backend = 🔥

The combo below allows to conveniently visualize large time_series data

In [ ]:
register_plotly_resampler(mode="auto", default_n_shown_samples=1500)
# pd.options.plotting.backend = 'plotly'


In [ ]:
df = pd.DataFrame(data={"sine": noisy_sine, "neg-sine": -noisy_sine}, copy=False)
df


In [ ]:
# TODO: for some reason pandas plotting backend + datetime index work really slow
# df.index = pd.date_range(start="2020-01-01", periods=len(df), freq="1s")

In [ ]:
df.plot(backend="plotly")


In [ ]:
unregister_plotly_resampler()

## **manual mode**: Basic sine example



✅ **advantages**:
* Highly configurable (e.g., downsampler per trace, number of shown sample per trace)
* Most optimizable (by leveraging the `hf_` arguments)

❌ **disadvantages**:
* more tedious (more code needs to be written)

This example shows how the `FigureResampler` can be used to efficiently construct a graph

In [ ]:
# 1. Wrap the figure with the FigureResampler class
fig = FigureResampler(go.Figure())

# Add the trace using the `hf_x` & `hf_y` for faster rendering
# (this does not impact the resampling speed once rendered)
fig.add_trace(go.Scattergl(showlegend=True), hf_x=x, hf_y=noisy_sine, max_n_samples=300)
# fig.add_trace(go.Scattergl(x=x, y=noisy_sine))

# Optional: update the layout
fig.update_layout(height=400, template="plotly_dark")

# 2. Call show_dash
fig.show_dash(mode="inline")


For the `FigureWidgetResampler` use case, you only need to chance way of displaying the figure

In [ ]:
# Same content as above, but using the FigureWidgetResampler class
fig = FigureWidgetResampler(go.Figure())
fig.add_trace(go.Scattergl(showlegend=True), hf_x=x, hf_y=noisy_sine, max_n_samples=900)
fig.update_layout(height=400, template="plotly_dark")

# Displaying the figure -> instead of `show_dash`, use IPython's `display`
display(fig)


## Advanced stuff

### Adjusting the data of your plotly-resampler figure at runtime

The `hf_data` property of the `FigureResampler` / `FigureWidgetResampler` can be used to change figure data at runtime

#### `FigureResampler`

In [ ]:
fig = FigureResampler(go.Figure())
fig.add_trace(go.Scattergl(showlegend=True, name="noisy_sine"), hf_y=noisy_sine)
display(fig.hf_data)
fig.show_dash(mode="inline")


now we adjust the figure data  
**note**: after running the cell below, we need to manually trigger a graph update (by for example zooming / resetting the axes) to ensure that the new data is shown.

In [ ]:
fig.hf_data[0]["y"] = -10 * noisy_sine
# make sure to interact win the figure to see the change


#### `FigureWidgetResampler`

In [ ]:
fig = FigureWidgetResampler(go.Figure())
fig.add_trace(go.Scattergl(showlegend=True, name="noisy_sine"), hf_y=noisy_sine)
display(fig.hf_data)
display(fig)


now we adjust the figure data  
**note**: after running the cell below, we need to manually trigger a graph update (by for example zooming / resetting the axes) to ensure that the new data is shown.

In [ ]:
fig.hf_data[0]["y"] = 10 * noisy_sine**2


**pro tip**: `FigureWidgetResampler` has the `reload_data` and `reset_axes` methods to do this automatically

In [ ]:
fig.hf_data[0]["y"] = -10 * noisy_sine**2
# the reload data function keeps the current zoom level
fig.reload_data()


In [ ]:
fig.hf_data[0]["y"] = 10 * noisy_sine**.1
# the reset axes
fig.reset_axes()


---

# Various other examples

The visualizations below shows how you plotly-resampler is used for various visualization configurations.

## The example `.gif` from the docs/README

Note how:
* The example figure withholds both time-indexed data and numeric-indexed data
* The power-consumption data uses `2,000` samples per trace (`max_n_samples`), whilst the other signals use the default of `1,000` samples per trace.
* The swimming pool trace uses another downsampling method (`EveryNthPoint`)
* The swimming pool trace uses `hf_hovertext` which shows the mean # of pool visitors of the last hour. 


In [ ]:
# ------------ loading the data -----------
df_gusb = pd.read_parquet("data/df_gusb.parquet")
df_data_pc = pd.read_parquet("data/df_pc_test.parquet")

# Create a noisy sine
n = 110_000_00  # 0
x = np.arange(n)
noisy_sine = (3 + np.sin(x / 200_000) + np.random.randn(n) / 10) * x / 100_000


In [ ]:
fig = FigureResampler(
    make_subplots(
        rows=2,
        cols=2,
        specs=[[{}, {}], [{"colspan": 2}, None]],
        subplot_titles=("GUSB swimming pool", "Generated sine", "Power consumption"),
        vertical_spacing=0.12,
    ),
    default_n_shown_samples=1000,
    verbose=False,
)


# ------------ swimming pool data -----------
df_gusb_pool = df_gusb["zwembad"].last("4D").dropna()
fig.add_trace(
    go.Scattergl(
        x=df_gusb_pool.index,
        y=df_gusb_pool.astype("uint16"),
        mode="markers",
        marker_size=5,
        name="occupancy",
        # showlegend=False,
    ),
    hf_hovertext="mean last hour: "
    + df_gusb_pool.rolling("1h").mean().astype(int).astype(str),
    downsampler=EveryNthPoint(interleave_gaps=False),
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Occupancy", row=1, col=1)


# ----------------- generated sine -----------
fig.add_trace(
    go.Scattergl(name="sin", line_color="#26b2e0"),
    hf_x=x,
    hf_y=noisy_sine,
    row=1,
    col=2,
)

# ------------- Power consumption data -------------
df_data_pc = df_data_pc.last("190D")
for i, c in enumerate(df_data_pc.columns):
    fig.add_trace(
        go.Scattergl(name=f"R-{i+1}", line_width=1),
        hf_x=df_data_pc.index,
        hf_y=df_data_pc[c],
        max_n_samples=2_000,  # The power consumption data uses 2,000 samples per trace
        row=2,
        col=1,
    )

fig.update_layout(height=650)
fig.update_yaxes(title_text="Watt/hour", row=2, col=1)
fig.update_layout(
    title="<b>Plotly-Resampler demo</b>",
    title_x=0.5,
    legend_traceorder="normal",
    legend=dict(orientation="h", y=1.11, xanchor="left", x=0),
)

fig.show_dash(mode="inline", debug=True, port=9029)


## Converting a `go.Figure`, with its traces, into a `FigureResampler`

This example first creates the `.gif` figure (with less data, otherwise the graph construction time would be too long) and then uses the  `convert_existing_traces` argument of the FigureResampler constructor to convert this into a FigureResampler figure.

In [ ]:
from plotly_resampler import unregister_plotly_resampler
unregister_plotly_resampler()

In [ ]:
df_gusb = pd.read_parquet(f"data/df_gusb.parquet")
df_data_pc = pd.read_parquet(f"data/df_pc_test.parquet")

n = 110_000
x = np.arange(n)
noisy_sine = (3 + np.sin(x / 2_000) + np.random.randn(n) / 10) * x / 2_000

# construct a normal figure object instead of a figureResample object
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{}, {}], [{"colspan": 2}, None]],
    subplot_titles=("GUSB swimming pool", "Generated sine", "Power consumption"),
    vertical_spacing=0.12,
)


# ------------ swimming pool data -----------
df_gusb_pool = df_gusb["zwembad"].last("4D").dropna()
fig.add_trace(
    go.Scattergl(
        x=df_gusb_pool.index,
        y=df_gusb_pool,  # .astype("uint16"),
        mode="markers",
        marker_size=5,
        name="occupancy",
        showlegend=True,
        hovertext="mean last hour: "
        + df_gusb_pool.rolling("1h").mean().astype(int).astype(str),
    ),
    # downsampler=EveryNthPoint(interleave_gaps=False),
    row=1,
    col=1,
)
fig.update_yaxes(title_text="Occupancy", row=1, col=1)


# ----------------- generated sine -----------
fig.add_trace(
    go.Scattergl(name="sin", line_color="#26b2e0", x=x, y=noisy_sine),
    row=1,
    col=2,
)

# ------------- Power consumption data -------------
df_data_pc = df_data_pc.last("50D")
for i, c in enumerate(df_data_pc.columns):
    fig.add_trace(
        go.Scattergl(
            name=f"room {i+1}",
            x=df_data_pc.index,
            y=df_data_pc[c],
        ),
        row=2,
        col=1,
    )

fig.update_layout(height=600)
fig.update_yaxes(title_text="Watt/hour", row=2, col=1)
fig.update_layout(
    title="<b>Plotly-Resampler demo - fig base</b>",
    title_x=0.5,
    legend_traceorder="normal",
    legend=dict(orientation="h", y=1.11, xanchor="left", x=0),
)


Note how the `data` property shape is the raw data size

In [ ]:
fig.data[1]["x"].shape


In [ ]:
# Convert the figure into a figurResampler figure by decorating it
fr_fig = FigureResampler(fig, default_n_shown_samples=500, convert_existing_traces=True)
fr_fig.show_dash(mode="inline")


In [ ]:
print("aggregated data    ", fr_fig.data[1]["x"].shape)
print("raw data (hf_data) ", fr_fig.hf_data[1]["x"].shape)

# The data-shape of the original figure stil remains the same
fig.data[1]["x"].shape

**Note**:
* the data size of `fr_fig` is reduced to `default_n_shown_samples` (_500_) but the original `fig` data is still equal to `110_000`.
* the raw data can be accessed and altered using the `hf_data` of the FigureResampler object ⬇️


In [ ]:
# We flip and take the sqarquere root of the data
fr_fig.hf_data[1]["y"] = -(noisy_sine**2)

# Make sure to interact witht he `Generated sine` subplot to see the changes

## Skin conductance example

This example is especially interesting as it **uses a _background-color_ to indicate a signal quality**.

To ensure the consistency of this background color use; we set `interleave_gaps` to False for the signal quality trace its downsampler.


In [ ]:
df_gsr = pd.read_parquet("data/processed_gsr.parquet")


In [ ]:
fig = FigureResampler(
    make_subplots(
        rows=2,
        cols=1,
        specs=[[{"secondary_y": True}], [{}]],
        shared_xaxes=True,
    ),
    default_n_shown_samples=1_000,
    verbose=False,
)
fig.update_layout(height=600, title="skin conductance example", title_x=0.5)


# -------------------------------- ROW 1 --------------------------------
# Add the skin conductance signals
for c in ["EDA", "EDA_lf_cleaned", "EDA_lf_cleaned_tonic"]:
    fig.add_trace(go.Scattergl(name=c), hf_x=df_gsr.index, hf_y=df_gsr[c], row=1, col=1)


# Show the Skin Conductance Response Peaks (SCR)
df_peaks = df_gsr[df_gsr["SCR_Peaks_neurokit_reduced_acc"] == 1]
fig.add_trace(
    trace=go.Scattergl(
        x=df_peaks.index,
        y=df_peaks["EDA_lf_cleaned"],
        visible="legendonly",
        mode="markers",
        marker_symbol="cross",
        marker_size=15,
        marker_color="red",
        name="SCR peaks",
    ),
    # Set limit_to_view to true so that the peaks disappear when out-of view-range
    # and thus not disturb the autoscale!!!
    limit_to_view=True,
)


# Display the skin conductance Signal Quality as background
df_grouped = groupby_consecutive(df_gsr["EDA_SQI"])
df_grouped["EDA_SQI"] = df_grouped["EDA_SQI"].map(bool)
df_grouped["good_sqi"] = df_grouped["EDA_SQI"].map(int)
df_grouped["bad_sqi"] = (~df_grouped["EDA_SQI"]).map(int)
for sqi_col, col_or in [
    ("good_sqi", "#2ca02c"),
    ("bad_sqi", "#d62728"),
]:
    fig.add_trace(
        go.Scattergl(
            x=df_grouped["start"],
            y=df_grouped[sqi_col],
            mode="lines",
            line_width=0,
            fill="tozeroy",
            fillcolor=col_or,
            opacity=0.1 if "good" in sqi_col else 0.2,
            line_shape="hv",
            name=sqi_col,
            showlegend=False,
        ),
        # The most important thing here is that interleave gaps is set to True
        # Additionally, the limit-to-view also ensures that the autoscale is not
        # disturbed.
        # downsampler=EveryNthPoint(interleave_gaps=False),
        limit_to_view=True,
        secondary_y=True,
    )


# -------------------------------- ROW 2 --------------------------------
# show the phasic EDA component
fig.add_trace(
    go.Scattergl(name="EDA_Phasic", visible="legendonly"),
    hf_x=df_gsr.index,
    hf_y=df_gsr["EDA_Phasic"],
    row=2,
    col=1,
)

fig.show_dash(mode="inline", port=9022)


## Categorical series - box & histogram

This example highlights how `plotly-resampler` supports combining high-frequency trace-subplots 
with non-scatterlike traces such as a histogram & a boxplot.

In [ ]:
# Create a categorical series, with mostly a's, but a few sparse b's and c's
cats_list = np.array(list("aaaaaaaaaa" * 1000))
cats_list[np.random.choice(len(cats_list), 100, replace=False)] = "b"
cats_list[np.random.choice(len(cats_list), 50, replace=False)] = "c"
cat_series = pd.Series(
    cats_list,
    dtype="category",
)

_nb_samples = 30_000
x = np.arange(_nb_samples).astype(np.uint32)
y = np.sin(x / 300).astype(np.float32) + np.random.randn(_nb_samples) / 5
float_series = pd.Series(index=x, data=y)


In [ ]:
base_fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{}, {}], [{"colspan": 2}, None]],
)
fig = FigureResampler(base_fig, default_n_shown_samples=1000, verbose=False)

fig.add_trace(
    go.Scattergl(name="cat_series"),
    hf_x=cat_series.index,
    hf_y=cat_series,
    row=1,
    col=1,
    hf_hovertext="text",
)

fig.add_trace(go.Box(x=float_series.values, name="float_series"), row=1, col=2)
fig.add_trace(go.Box(x=float_series.values**2, name="float_series**2"), row=1, col=2)

# add a not hf-trace
fig.add_trace(
    go.Histogram(
        x=float_series,
        name="float_series",
    ),
    row=2,
    col=1,
)
fig.show_dash(mode="inline", port=9032)
